In [11]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [12]:
# loading the images and preprocessing

# initialize empty list for data(images) and labels
data = []
labels = []


print("[INFO] loading images...")
imagePaths = list(paths.list_images("Mask_Dataset/"))  # a list of paths of all images in the dataset 

# loop over the image paths
for imagePath in imagePaths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]  # check wheter the image is inside with_mask or without_mask folder
    # load the input image (224x224) and preprocess it
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)
# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

print("[INFO] images are loaded successfully!...")

[INFO] Loading images...
[INFO] Images are loaded successfully!...


In [13]:
lb = LabelBinarizer() # Binarize labels in a one-vs-all fashion
labels = lb.fit_transform(labels) # Fit label binarizer and transform multi-class labels to binary labels.
labels = to_categorical(labels) #Converts a class vector (integers) to binary class matrix.

# partition the data into training and testing splits 
# 80% of data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)

# generate argumented images for training
aug = ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,fill_mode="nearest")


In [14]:
# load the MobileNetV2 network, ensuring the head FC layer sets are left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model 
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


C:\Users\DELL\miniconda3\envs\tensorflow\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [25]:
INIT_LR = 1e-4 # initial learning rate
EPOCHS = 20    #number of epochs
BS = 32  # batch size

# compile the model 
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])



# train the head of the network
print("[INFO] training head...")
H = model.fit(aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

[INFO] compiling model...
[INFO] training head...
Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
37/37 [==============================] - 1646s 44s/step - loss: 0.5751 - acc: 0.7174 - val_loss: 0.2967 - val_acc: 0.9007
Epoch 2/20
37/37 [==============================] - 253s 7s/step - loss: 0.3455 - acc: 0.8562 - val_loss: 0.1911 - val_acc: 0.9603
Epoch 3/20
37/37 [==============================] - 252s 7s/step - loss: 0.2776 - acc: 0.8936 - val_loss: 0.1623 - val_acc: 0.9669
Epoch 4/20
37/37 [==============================] - 251s 7s/step - loss: 0.2368 - acc: 0.8962 - val_loss: 0.1195 - val_acc: 0.9768
Epoch 5/20
37/37 [==============================] - 252s 7s/step - loss: 0.1999 - acc: 0.9166 - val_loss: 0.1043 - val_acc: 0.9801
Epoch 6/20
37/37 [==============================] - 252s 7s/step - loss: 0.1646 - acc: 0.9328 - val_loss: 0.1257 - val_acc: 0.9636
Epoch 7/20
37/37 [==============================] - 250s 7s/step - l

In [29]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)


# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)


# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,target_names=lb.classes_))

# saving the model into disk
print("[INFO] saving mask detector model...")
model.save('mask_detector.model', save_format="h5")

[INFO] evaluating network...
                      precision    recall  f1-score   support

   Dataset/with_mask       0.98      0.99      0.99       151
Dataset/without_mask       0.99      0.98      0.99       151

            accuracy                           0.99       302
           macro avg       0.99      0.99      0.99       302
        weighted avg       0.99      0.99      0.99       302

[INFO] saving mask detector model...
